In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv')
print('train', train.shape)
print('test', test.shape)
train.head()

train (26570, 26)
test (20775, 25)


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
!pip install pycaret --ignore-installed llvmlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 25.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 33.7 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.9 MB/

In [5]:
from pycaret.classification import *

In [6]:
train.columns

Index(['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17',
       'failure'],
      dtype='object')

###Feature Engineering

Ambrosm suggested in his notebook that feature measurement_3 has lower failure rate than average failure rate and measurement_5 has higher failure rate than average failure rate so it can be inferred that availability or unavailability of these features is directly correlated with the failure so we can add availablity of these two features as new features.

In [9]:
train['m_3_missing']=train.measurement_3.isna().astype(int)
train['m_5_missing']=train.measurement_5.isna().astype(int)

test['m_3_missing']=test.measurement_3.isna().astype(int)
test['m_5_missing']=test.measurement_5.isna().astype(int)

In [10]:
train.describe(include = ['category', 'object'])

,product_code,attribute_0,attribute_1
count,26570,26570,26570
unique,5,2,3
top,C,material_7,material_8
freq,5765,21320,10865


In [11]:
train.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,m_3_missing,m_5_missing
0,0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0,0,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0,0,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0,0,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0,0,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0,0,0


In [12]:
clf_1 = setup(data = train, target = 'failure', session_id = 100, 
              normalize = True,
              transformation = True, 
              remove_multicollinearity = True,
              multicollinearity_threshold = 0.85,
              remove_outliers = True,
              handle_unknown_categorical = True,
              ignore_low_variance = True,
              imputation_type = 'iterative',
              numeric_iterative_imputer = 'lightgbm',
              categorical_iterative_imputer = 'lightgbm',
              categorical_imputation = 'mode',
              unknown_categorical_method = 'most_frequent',
              fix_imbalance = True,
              feature_interaction = True,
              interaction_threshold = 0.1,
              create_clusters = True,
              feature_selection = True,
              feature_selection_threshold = 0.7,
              log_experiment = True,
              data_split_stratify = True,
              fold = 5,
              n_jobs = -1,
              use_gpu = True,
              silent = True,
              categorical_features = ['attribute_0', 'attribute_1'],
              numeric_features = ['loading',
       'attribute_2', 'attribute_3',],
              ignore_features = ['id', 'product_code', 'measurement_0', 'measurement_1',
       'measurement_2',  'measurement_4', 
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17' ]) # session_id equivalent to random_state

,Description,Value
0,session_id,100
1,Target,failure
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(26570, 28)"
5,Missing Values,True
6,Numeric Features,5
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


2022/08/15 14:49:37 INFO mlflow.tracking.fluent: Experiment with name 'clf-default-name' does not exist. Creating a new experiment.


PicklingError: Can't pickle <class 'pyod.models.pca.PCA'>: it's not found as pyod.models.pca.PCA

In [13]:
#types of models available in the library
models()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [14]:
blender_soft = blend_models(compare_models(include = ['lr', 'nb', 'rf', 'lightgbm', 'ridge', 'ada', 'xgboost'], n_select = 3, 
                                           sort = 'AUC'),
                            method = 'soft', weights = [0.5,0.3,0.2],
                            choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5447,0.5710,0.5532,0.2463,0.3408,0.0657,0.0783
1,0.5515,0.5811,0.5785,0.2554,0.3544,0.0839,0.1005
2,0.5405,0.5761,0.5798,0.2500,0.3494,0.0740,0.0897
3,0.5386,0.5478,0.5193,0.2351,0.3237,0.0438,0.0518
4,0.5853,0.5709,0.4934,0.2550,0.3362,0.0772,0.0861
Mean,0.5521,0.5694,0.5448,0.2484,0.3409,0.0689,0.0813
Std,0.0172,0.0114,0.0338,0.0074,0.0107,0.0139,0.0164


In [15]:
blender_soft

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=100, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
!pip install tune-sklearn ray[tune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 955.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.1 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.5
    Uninstalling protobuf-4.21.5:
      Successfully uninstalled protobuf-4.21.5
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
tfx-bsl 1.9.0 req

In [18]:
tuned_blend = tune_model(blender_soft, early_stopping = True, early_stopping_max_iters = 15, 
                        search_library = 'tune-sklearn', search_algorithm = 'random', tuner_verbose = 5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5515,0.5837,0.5598,0.2513,0.3469,0.0753,0.0894
1,0.5467,0.5884,0.5864,0.2546,0.3551,0.0829,0.1002
2,0.5654,0.5907,0.5638,0.2598,0.3557,0.0908,0.1064
3,0.5443,0.5588,0.5433,0.2436,0.3364,0.0606,0.0720
4,0.5618,0.5933,0.5638,0.2579,0.3539,0.0873,0.1027
Mean,0.5539,0.5830,0.5634,0.2535,0.3496,0.0794,0.0941
Std,0.0083,0.0125,0.0138,0.0057,0.0073,0.0107,0.0124


In [19]:
print(tuned_blend)

LogisticRegression(C=0.08, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=100, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [23]:
evaluate_model(tuned_blend)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [22]:
# predictions on validation set
predict_model(tuned_blend)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5495,0.5798,0.5764,0.2538,0.3524,0.0811,0.0972


,data_cluster_2,attribute_1_material_8,data_cluster_3,loading,measurement_3,attribute_1_material_6,attribute_0_material_5,attribute_3,attribute_1_material_5,attribute_2,...,attribute_1_material_6_multiply_loading,attribute_2_multiply_measurement_5,attribute_0_material_5_multiply_measurement_3,attribute_2_multiply_measurement_3,loading_multiply_data_cluster_3,loading_multiply_measurement_5,loading_multiply_data_cluster_2,failure,Label,Score
0,1.0,1.0,0.0,0.352689,-1.377007,0.0,0.0,-1.394399,0.0,1.388293,...,0.0,1.455552,-0.000000,-1.911689,0.000000,0.369775,0.352689,0,1,0.5569
1,0.0,0.0,1.0,-0.436922,-0.251811,0.0,0.0,-0.934828,1.0,-0.392943,...,-0.0,0.024194,-0.000000,0.098947,-0.436922,0.026902,-0.000000,0,0,0.5355
2,0.0,0.0,1.0,-1.212272,0.767545,0.0,0.0,-0.934828,1.0,-0.392943,...,-0.0,-0.538460,0.000000,-0.301601,-1.212272,-1.661209,-0.000000,0,0,0.5875
3,0.0,0.0,0.0,-0.499453,1.356423,0.0,1.0,0.404144,1.0,0.925445,...,-0.0,0.336535,1.356423,1.255294,-0.000000,-0.181625,-0.000000,0,0,0.5495
4,0.0,1.0,0.0,-1.120098,0.709148,0.0,0.0,0.404144,0.0,-1.352000,...,-0.0,-1.249459,0.000000,-0.958768,-0.000000,-1.035145,-0.000000,0,0,0.5792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7967,1.0,1.0,0.0,1.866052,1.225579,0.0,0.0,-1.394399,0.0,1.388293,...,0.0,0.827480,0.000000,1.701463,0.000000,1.112244,1.866052,0,1,0.6824
7968,0.0,0.0,1.0,-0.170362,-1.023024,0.0,0.0,-0.934828,1.0,-0.392943,...,-0.0,0.130831,-0.000000,0.401990,-0.170362,0.056722,-0.000000,0,0,0.5170
7969,0.0,1.0,0.0,0.281758,1.215514,0.0,0.0,0.404144,0.0,-1.352000,...,0.0,1.308555,0.000000,-1.643375,0.000000,-0.272704,0.000000,0,1,0.5006
7970,1.0,1.0,0.0,0.430630,-1.283209,0.0,0.0,-1.394399,0.0,1.388293,...,0.0,0.971792,-0.000000,-1.781471,0.000000,0.301437,0.430630,0,1,0.5547


In [25]:
final_blend =finalize_model(tuned_blend)

In [26]:
print(final_blend)

LogisticRegression(C=0.08, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=100, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [27]:
predictions = predict_model(final_blend, data = test)
predictions.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,m_3_missing,m_5_missing,Label,Score
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,13.772,13.659,16.825,13.742,17.710,634.612,0,0,1,0.5336
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,12.473,17.468,16.708,14.776,14.102,537.037,0,0,0,0.5039
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,10.907,13.363,15.737,17.065,16.021,658.995,0,0,1,0.5002
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,10.933,15.501,15.667,12.620,16.111,594.301,0,0,0,0.5304
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,11.941,16.070,16.183,13.324,17.150,801.044,0,0,1,0.6295


In [28]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['failure'] = predictions['Label']
submission.to_csv("submission.csv", index = False)

In [29]:
save_model(final_blend, 'Naive_Bayes_blend_feature_engg_15sep2022_2040')

PicklingError: Can't pickle <class 'pyod.models.pca.PCA'>: it's not found as pyod.models.pca.PCA